In [ ]:
# Author: Will Connell 

In [1]:
!pip install scanpy
!pip install scperturb
!pip install scikit-misc


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for scperturb from https://files.pythonhosted.org/packages/0b/96/1f1cdce4b007c24cb7171cf368ced8f1e6723f7132bf0e3afb43cee5858d/scperturb-0.1.0-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# datastes here: http://projects.sanderlab.org/scperturb/datavzrd/scPerturb_vzrd_v1/dataset_info/index_1.html

!wget https://zenodo.org/record/7041849/files/NormanWeissman2019_filtered.h5ad

zsh:1: command not found: wget


In [20]:
import scanpy as sc
import anndata as ad
from scipy.sparse import csr_matrix

from scperturb import *

import scanpy as sc
import numpy as np
import pandas as pd

In [21]:
'''
See here: https://github.com/sanderlab/scPerturb/blob/f64821e5e621b3f6147b47f318ae1775d4eff324/revision/snakemake/Snakefile#L177-L215
'''

def equal_subsampling(adata, obs_key, N_min=None):
    '''
    Subsample to same class sizes. Classes given by obs_key pointing to categorical in adata.obs.
    If N_min is given, downsamples to at least this number instead of the number of cells in the smallest class
    and throws out classes with less than N_min cells.
    '''
    counts = adata.obs[obs_key].value_counts()
    groups = counts.index[counts >= N_min]  # ignore groups with less than N_min cells to begin with

    if len(groups) == 0:
        raise ValueError(f"No groups have at least {N_min} cells.")

    # We select downsampling target counts by min-max, i.e.
    # the largest N such that every group has at least N cells before downsampling.
    N = np.min(counts[groups])
    N = max(N, N_min) if N_min else N

    # subsample indices per group
    indices = [
        np.random.choice(
            adata.obs_names[adata.obs[obs_key] == group], size=N, replace=False
        )
        for group in groups
    ]
    selection = np.hstack(np.array(indices))
    return adata[selection].copy()

In [22]:
# Load your anndata object
adata = sc.read_h5ad("NormanWeissman2019_filtered.h5ad")

In [23]:
adata

AnnData object with n_obs × n_vars = 111445 × 33694
    obs: 'guide_id', 'read_count', 'UMI_count', 'coverage', 'gemgroup', 'good_coverage', 'number_of_cells', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'perturbation', 'nperts', 'ngenes', 'ncounts', 'percent_mito', 'percent_ribo'
    var: 'ensemble_id', 'ncounts', 'ncells'

In [24]:
total_unique_perts = adata.obs['perturbation'].nunique()
total_unique_perts

237

In [25]:
# filter all the 'perturbation's that contain '_' from the a data object
adata = adata[~adata.obs['perturbation'].str.contains('_')]
total_single_gene_perts = adata.obs['perturbation'].nunique()
print("Number of single gene perturbations: ", total_single_gene_perts - 1)

Number of single gene perturbations:  105


In [26]:
# Basic QC and preprocessing
print('Shape before filtering: ', adata.shape)
sc.pp.filter_cells(adata, min_counts=1000)
print('Number of perturbations: ', adata.obs['perturbation'].nunique())

# Apply equal subsampling
adata = equal_subsampling(adata, 'perturbation', N_min=300)
print('Number of perturbations after subsampling: ', adata.obs['perturbation'].nunique())

adata.layers['counts'] = adata.X.copy()
print('Normalizing counts...')
sc.pp.normalize_per_cell(adata)
sc.pp.filter_genes(adata, min_cells=50)
print('Applying log1p...')
sc.pp.log1p(adata)
print('Shape after filtering: ', adata.shape)

# Select HVGs
print('Selecting highly variable genes...')
sc.pp.highly_variable_genes(adata, n_top_genes=2000,
                            subset=False, flavor='seurat_v3', layer='counts')
print('Number of perturbations after selecting HVGs: ', adata.obs['perturbation'].nunique())

print('Running PCA...')
sc.pp.pca(adata, use_highly_variable=True, n_comps=50)

# Save the filtered and processed anndata object
# adata.write_h5ad('filtered_anndata.h5ad')

# Display the final number of cells per perturbation
cell_counts = adata.obs['perturbation'].value_counts()
print(cell_counts)

Shape before filtering:  (69686, 33694)


/Users/aaronw/Desktop/PhD/Research/QMUL/Research/scBench/venv/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:161: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_counts"] = number


Number of perturbations:  106
Normalizing counts...
Applying log1p...
Shape after filtering:  (25755, 14811)
Selecting highly variable genes...
Running PCA...


/Users/aaronw/Desktop/PhD/Research/QMUL/Research/scBench/venv/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:374: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)


perturbation
AHR        303
OSR2       303
RUNX1T1    303
RREB1      303
RHOXF2     303
          ... 
FEV        303
ETS2       303
ELMSAN1    303
EGR1       303
control    303
Name: count, Length: 85, dtype: int64


In [28]:
df = etest(adata, runs=10000, n_jobs=-1)

print(df.shape)
df['significant_adj'].sum() / len(df)
# save the df results
df.to_csv('etest_results_single_perts.csv')


100%|██████████| 10000/10000 [07:51<00:00, 21.23it/s]


(85, 5)


/Users/aaronw/Desktop/PhD/Research/QMUL/Research/scBench/venv/lib/python3.10/site-packages/statsmodels/stats/multitest.py:186: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


In [ ]:
df.head()

(153, 5)


,edist,pvalue,significant,pvalue_adj,significant_adj
AHR,21.986925,0.0001,True,0.015184,True
AHR_KLF1,35.566693,0.0001,True,0.015184,True
ARRDC3,6.685922,0.0001,True,0.015184,True
ATL1,57.990839,0.0001,True,0.015184,True
BAK1,0.725031,0.0282,True,0.082237,False


In [29]:
# open etest results
df = pd.read_csv('etest_results_single_perts.csv')
df

,Unnamed: 0,edist,pvalue,significant,pvalue_adj,significant_adj
0,AHR,21.757839,0.0001,True,0.008464,True
1,ARRDC3,6.931206,0.0001,True,0.008464,True
2,ATL1,60.535701,0.0001,True,0.008464,True
3,BAK1,0.994366,0.0104,True,0.030877,True
4,BCL2L11,0.082736,0.2923,False,0.499161,False
...,...,...,...,...,...,...
80,ZBTB1,27.508356,0.0001,True,0.008464,True
81,ZBTB25,13.570649,0.0001,True,0.008464,True
82,ZC3HAV1,3.710965,0.0001,True,0.008464,True
83,ZNF318,2.619759,0.0001,True,0.008464,True
